# Homework 5 Random Forests and Decision Trees.

In [ ]:
import numpy as np
import math
from scipy import stats

Let's implement a Decision tree using Shannon Entropy and expected information gain.
To do this we need to
* Implement functions getting information entropy
* Implement a decision tree class which acts on a "labeled" dataset

## Information Theory

In [ ]:
# Based on the standard definition of entropy.
def entropy(data, classes):
    entr = 0
    for cls in classes:
        probi = len(cls)/len(data)
        entr += -probi*math.log2(probi)
    
    return entr

# Determines the information gain for an attribute for a data set.
def discrete_igain(data, classes, a, possible_a):
    data_entropy = entropy(data,classes)
    return discrete_igain_fast(data,classes,a,possible_a,data_entropy)

def discrete_igain_fast(data,classes, a , possible_a, data_entropy):
    mutualinf = 0
    for attr in possible_a:
        data_attr = [x for x in data if x[a] == attr]
        mutualinf += len(data_attr)/len(data)*entropy(data_attr, classes)
    
    return  data_entropy - mutualinf

#not an efficient classes algorithm, but no fucks given; only use once.
def get_classes(data, labels):
    cls = []
    label_vals = np.unique(labels)
    for label in label_vals:
        cls.append([x for x,y in zip(data,labels) if y == label])
        
    return cls

def get_attrvals(data, a):
    return np.unique(np.transpose(data)[a])

### Tests

In [ ]:
#XOR!
x = np.array([[0,0],
              [1,0],
              [0,1],
              [1,1],
              [-1,1],
              [-1,0],
              [-1,-1]])
y = np.array([0,1,1,0,0,0,1])

cls = get_classes(x,y)
entropy(x, cls)
discrete_igain(x, cls, 1, get_attrvals(x, 1))

In [ ]:
def submap(data,label, restriction):
    indices, constraints = zip(*restriction)
    
    a,b = zip(*[(x,y) for x,y in zip(data,label) if x[indices] == constraints])
    return (np.array(list(a)), np.array(list(b)))
xp, yp =submap(x,y, [(0,-1)])
clsp = get_classes(xp,yp)

def tabstr(n):
    retstr = ""
    for i in range(n):
        retstr += "\t"
        
    return retstr

## Discrete Decision Trees
Since we've built our tests and informtion theoretic methods; it is now probably a good idea to build a decision tree class. This should attempt to maximize information gain upon construction and thereafter be able to classify
any given training example confined to the attributional classes of the dataset.

After we do this, we'll define a decision tree which can take discrete and continuous vales, specified during construction.

In [ ]:
# A toy example of the discrete decision tree
class ddtree:
    ### gets decisional
    def __init__(self, data, labels, restricted=[]):
        self.data = data
        self.labels = labels
        self.classes = get_classes(data,labels)
        self.subtrees = {}
        self.entropy = entropy(self.data, self.classes)
        self.attr = None
        self.restricted = restricted
    
    #trains the tree:
    def train(self):
        #check to see that the label set is unique
        if len(np.unique(self.labels)) <= 1:
            return
        
        #Get the igains for all of the attributes.
        igains = []
        
        for i in range(self.data.shape[1]): #random forest would take a random sub sample.
            #make sure we don't consider all the values a previous node has considered.
            if self.restricted is None or i not in self.restricted:
                atr_vals = get_attrvals(self.data,i)
                
                if len(atr_vals) >1: #We only want to consider attributes whose possible values are different
                    igains.append(discrete_igain(self.data, self.classes, i, atr_vals))
                
        #Best attribute is the argmax
        best_attr = np.argmax(igains)
        self.attr = best_attr
        
        # restrict the attributes which the trees can consider!
        subres = [best_attr]
        subres.extend(self.restricted)
        
        #Make the sub decision trees for each choice of the attribute.
        for val in get_attrvals(self.data,best_attr):
            #make a subtree
            dp, lp = submap(self.data, self.labels, [(best_attr, val)])
            #make the subtree decisions based on if they satisfy a lambda; to abstract in dynamic trees.
            tree = ddtree(dp, lp, subres)
            self.subtrees[(val, lambda x, best_attr=best_attr, val=val: x[best_attr] == val)] = tree

            
        #train all of the trees
        for (val, test), tree in self.subtrees.items():
            tree.train()
            
    
    def classify(self, x):
        if len(self.subtrees) > 0 and self.attr is not None:
            for (val, test), tree in self.subtrees.items():
                if test(x):
                    return tree.classify(x)
        else:
            return self.labels[0] #Return the only label in the tree.
        
    def print_tree(self, n):
        if len(self.subtrees) > 0 and self.attr is not None:
            retstr =  "x[" + str(self.attr) + "] subtrees(" + str(len(self.subtrees)) + ")\n" + tabstr(n)
            for (val, test), tree in self.subtrees.items():
                retstr += "- " +  str(val) + " ->"+"\t" + tree.print_tree(n+1) +"\n" + tabstr(n)
        else:
            return  "Y:" + str(self.labels[0])
        
        return retstr
        
    def __str__(self):
        return self.print_tree(0)
            

In [ ]:
test = ddtree(x,y)

In [ ]:
# Let's try training this thing!
test.train()
print(test)
print("Classification error!")
for i in range(len(x)):
    print(test.classify(x[i]), y[i])

# Full decision trees with continuous values and random subset selection!

In order to implement the full version of decsion trees, we don't think of attributes as elements of a set but in fact indicator functions of subsets. In the case of the `ddtree` we have $\chi_a$ for singleton sets $\{a\} := a$. For the full thing we'll just extend this notion to "axis aligned" intervals.

In [ ]:
#def indicator
class Indicator:
    def __init__(self, func, name):
        self.func = func
        self.name = name
    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)
    def __str__(self):
        return str(self.name)
    def __repr__(self):
        return str(self)



# make our indicator functions.
def dindicator(a):
    return Indicator((lambda x,a=a: x == a),  str(a))
def ci_indicator(a,b):
    return Indicator((lambda x,a=a,b=b: a <= x and x < b), "[" +str(a) + ", " + str(b) + ")")  
def cray_indicator(a, inf):
    if inf > 0:
        return Indicator((lambda x,a=a: a <= x), "(" + str(a) +", inf)")
    elif inf < 0:
        return Indicator((lambda x,a=a: x < a),  "(-inf," + str(a) +")")
    else:
        return dindicator(a)
    
    
def split_indicators(splits):
    indicators = []
    
    #make end rays
    indicators.append(cray_indicator(splits[0], -1))
    
    #make middle intervals
    for i, val in enumerate(splits[:-1]):
        indicators.append(ci_indicator(splits[i], splits[i+1]))
    #make end rays
    indicators.append(cray_indicator(splits[-1], 1))
    
    return indicators

def singleton_indicator(possible_vals):
    indicators = []
    for val in possible_vals:
        indicators.append(dindicator(val))
    
    return indicators

We need to now make the indicator versions of our helper functions.

In [ ]:
# Based on the standard definition of entropy.
def entropy(data, classes):
    
    if len(data) == 0:
        return 0
    entr = 0
    for cls in classes:
        cls = np.intersect1d(cls, data)
        probi = len(cls)/len(data)
        entr += -probi*math.log2(probi)
    
    return entr

# Determines the information gain for an attribute for a data set.
def igain(data, classes, attribute, indicators):
    data_entropy = entropy(data,classes)
    return igain_fast(data, classes, attribute, indicators, data_entropy)

def igain_fast(data, classes, attribute, indicators, data_entropy):
    mutualinf = 0
    for indicator in indicators:
        data_attr = [x for x in data if indicator(x[attribute])]
        mutualinf += len(data_attr)/len(data)*entropy(data_attr, classes)
    
    return  data_entropy - mutualinf

#not an efficient classes algorithm, but no fucks given; only use once.
def get_classes(data, labels):
    cls = []
    label_vals = np.unique(labels)
    for label in label_vals:
        cls.append([x for x,y in zip(data,labels) if y == label])
        
    return cls

def get_indicators(data, classes, a, discrete=True, heuristic=None):
    if discrete:
        return singleton_indicator(np.unique(np.transpose(data)[a]))
    else:
        if heuristic is None:
            return split_indicators([np.mean(np.transpose(data)[a])])
        else:
            return heuristic(data, classes, a, np.transpose(data)[a])

# Submaps data based on restrictions.
def submap(data,label, restriction):
    def satisfies(x, restriction=restriction):
        for index, constraint in restriction:
            if not constraint(x[index]):
                return False
        
        return True
    valueset = [(x,y) for x,y in zip(data,label) if satisfies(x)]
    if not valueset:
        return None, None
    else:
        a,b = zip(*valueset)
        return (np.array(list(a)), np.array(list(b)))

# Splitpoint Heuristics
We need heuristics for data deemed continuous. How should we split it?

In [ ]:
import random

#Breaks an interval up into n random splits
def get_random_splits(n, M, m):
        splits = []
        for i in range(n):
            splits.append(random.random()*(M-m) + m)
        splits.sort()
        return splits

def random_heuristic(n=None):
    def get_splits(data, classes, a, values, n=n):
        if n is None:
            n = math.floor(math.sqrt(len(values)))
        M,m = np.max(values), np.min(values)
        
        splits = get_random_splits(n, M, m)
        return split_indicators(splits)
    
    return get_splits

def entropy_heuristic():
    def get_splits(data, classes, a, values):
        possible_indicators = []
        values = np.unique(values)
        for value in values:
            possible_indicators.append(split_indicators([value]))
            
        
        igains = list(map(lambda x: igain(data, classes, a, x), possible_indicators))
        
        return possible_indicators[np.argmax(igains)]
    
    return get_splits

# Gets the most adventageous information gain according to splits
# TODO TEST:
def random_entropy_heuristic(n=None,p=100):
    def get_splits(data,classes, a, values, n=n, p=p):
        
        possible_indicators = []
        rando_splitter = random_heuristic(n)
        for i in range(p):
            possible_indicators.append(rando_splitter(data, classes,a, values))
        
        igains = map(lambda x: igain(data, classes, a, x), possible_indicators)
         
        
        return possible_indicators[np.argmax(igains)]
    
    return get_splits

# Decision Tree
Now that we've resolved the problem to acting on arbitrary indicator functions for subsets, we can consider building a general decision tree!

In [ ]:
# A toy example of the discrete decision tree
class dtree:
    ### gets decisional
    def __init__(self, data, labels, restricted=[], depth=0, maxdepth=None):
        self.data = data
        self.labels = labels
        self.classes = get_classes(data,labels)
        self.subtrees = {}
        self.entropy = entropy(self.data, self.classes)
        self.attr = None
        self.depth = depth
        self.restricted = restricted
        self.maxdepth = maxdepth

    def bag(self, attributes, bag):
        if bag > 0:
            return random.sample(attributes, min(bag, len(attributes)))
        else:
            return attributes
                                
                             
    #trains the tree:
    def train(self, bag=0, typetable=None, heuristic=None):
        #check to see that the label set is unique
        if len(np.unique(self.labels)) <= 1:
            return
        
        #Get the igains for all of the attributes.
        igains = []
        attr_indic = {}
        
        attributes = [ i for i in range(len(self.data[0])) if i not in self.restricted]
        attributes = self.bag( attributes, bag)
        
        
        if not attributes or self.depth == self.maxdepth:
            return #We've reached the end of our tree.
        
        for i in attributes: #random forest would take a random sub sample.
            #make sure we don't consider all the values a previous node has considered.
            if self.restricted is None or i not in self.restricted:
                discrete = False
                #Assume that all data is discrete :()
                if typetable is False:
                    discrete = False              
                elif typetable is None or typetable[i]:
                    discrete = True
                
                indicators = get_indicators(self.data, self.classes, i, discrete, heuristic)
                # In case this is the best indicator    
                attr_indic[i] = indicators
                
                #We only want to consider attributes whose possible values are different
                igains.append(igain(self.data, self.classes, i, indicators))
        
        
        #Best attribute is the argmax
        best_attr = attributes[np.argmax(np.array(igains))]
        self.attr = best_attr
        
        # restrict the attributes which the trees can consider!
        subres = [best_attr]
        subres.extend(self.restricted)
        
        
        #Make the sub decision trees for each choice of the attribute.
        for val in attr_indic[best_attr]:
            #make a subtree
            dp, lp = submap(self.data, self.labels, [(best_attr, val)])
            
            #Only make a subtree if there are datapoints which would even satisfy it!
            if dp is not None:
                #make the subtree decisions based on if they satisfy a lambda; to abstract in dynamic trees.
                tree = dtree(dp, lp, subres, depth=self.depth+1, maxdepth=self.maxdepth)
                self.subtrees[(val, lambda x, best_attr=best_attr, val=val: val(x[best_attr]))] = tree

            
        #train all of the trees
        for (val, test), tree in self.subtrees.items():
            tree.train(bag, typetable, heuristic)
            
    
    def classify(self, x):
        if len(self.subtrees) > 0 and self.attr is not None:
            for (val, test), tree in self.subtrees.items():
                if test(x):
                    return tree.classify(x)
        else:
            return stats.mode(self.labels)[0][0] #Return the only label in the tree.
        
    def print_tree(self, n):
        if len(self.subtrees) > 0 and self.attr is not None:
            retstr =  "x[" + str(self.attr) + "] subtrees(" + str(len(self.subtrees)) + ")\n" + tabstr(n)
            for (val, test), tree in self.subtrees.items():
                retstr += "- " + str(val) + " ->"+"\t" + tree.print_tree(n+1) +"\n" + tabstr(n)
        else:
            return  "Y:" + str(self.labels[0])
        
        return retstr
        
    def __str__(self):
        return self.print_tree(0)

Wow that was a lot. Okay, so if that worked; we should definitely be able to the discrete case! :e


In [ ]:
test = dtree(x,y) #Shit it worked!
test.train()
print(test)
print("Classification error!")
for i in range(len(x)):
    print(test.classify(x[i]), y[i])

Shit that worked. Let's try some continuous data.

In [ ]:
#XOR!
x = np.array([[0,0],
              [0,0.1],
              [0,0.2],
              [0,0.3],
              [0,0.4],
              [0,0.5],
              [0,0.6],
              [0,0.7],
              [0,0.8],
              [0,0.9],
              [0,1.0],])
y = np.array([0,0,0,0,0,1,1,1,1,1,1])

cls = get_classes(x,y)
typetable = {}
typetable[0] = True
typetable[1] = False

In [ ]:
test = dtree(x,y)
test.train(typetable=typetable, heuristic=entropy_heuristic())
print(test)
print("Classification error!")
for i in range(len(x)):
    print(test.classify(x[i]), y[i])

# Random Forests
Now we can aggregate our forest fun with random forests who decorolate decision trees and take therir average!

In [ ]:
# Not meant for regression!
class rforest:
    #initalizes all the trees and partitions the data.
    def __init__(self, data, labels, numtrees, partition_size, maxdepth=None):
        self.trees = []
        self.data = data
        self.labels = labels
        self.partitions = []
        
        #create partitions
        for i in range(numtrees):
            
            dpart, lpart = zip(*random.sample(list(zip(data,labels)), partition_size))
           
            self.partitions.append((dpart, lpart))
            self.trees.append(dtree(np.array(list(dpart)), np.array(list(lpart)), maxdepth=maxdepth))
        
        
    
    def train(self, bag=0, typetable=None, heuristic=None):
        for tree in self.trees:
            tree.train(bag, typetable, heuristic)
        
    
    #Ensemble voting
    def ensemble(self,x):
        box = {}
        for tree in self.trees:
            vote = tree.classify(x)
            if vote is not None:
                if vote in box:
                    box[vote] += 1
                else:
                    box[vote] = 1
        
        
        return box
    
    def classify(self, x):
        predictions = self.ensemble(x)
        i, m = None, -1
        for pred, votes in predictions.items():
            if votes > m:
                i,m = pred, votes
        
        return i

Okay now let's try this thing on that x,y dataset

In [ ]:
test = rforest(x,y, 6, 3)
test.train(heuristic=entropy_heuristic(), bag=1)
print("Classification error!")
for i in range(len(x)):
    print(test.ensemble(x[i]), y[i])

# Let's do Some fucking Spam

In [ ]:
import scipy.io
from sklearn.preprocessing import normalize

In [ ]:
spam_data = scipy.io.loadmat("../data/spam_dataset/spam_data.mat")
spam_train = spam_data['training_data']
spam_test = spam_data['test_data']

spam_train_data_raw =  spam_train
spam_train_label_raw =  spam_data['training_labels']

spam_tlabel = spam_train_label_raw.ravel()
div_train = [1.0/max(arr) for arr in spam_train_data_raw.T]
spam_tdata = spam_train_data_raw 
spam_test_data =  spam_test

#Shuffle that spam data good.
shuffle = np.random.permutation(np.arange(spam_tdata.shape[0]))
spam_tdata, spam_tlabel = spam_tdata[shuffle], spam_tlabel[shuffle]

#VALIDATION
spam_valid_data  =  spam_tdata[0:750]
spam_valid_label = spam_tlabel[0:750]

#TRAINING
spam_train_data  =spam_tdata[750:]
spam_train_label =spam_tlabel[750:]

In [ ]:
ents = rforest(spam_train_data, spam_train_label, 20, int(len(spam_train_data)/5), maxdepth=20)

In [ ]:
ents.train(typetable=False, heuristic=random_entropy_heuristic(3, 100),bag=20)

In [ ]:
print("Classification error!")
net_error = 0
for i in range(len(spam_valid_data)):
    pred = ents.classify(spam_valid_data[i]) 
    if pred is None:
        print(pred)
    if pred is not None:
        net_error += abs(pred - spam_valid_label[i])
    
print(net_error/len(spam_valid_data))

In [ ]:
spam_results = []


for i, dp in enumerate(spam_test_data):
    pred = ents.classify(dp) 
    if pred is None:
        pred = 0
    spam_results.append(np.array([i+1, pred]))

np.savetxt(
    'kagglespam.csv',           # file name
    spam_results,                # array to save
    fmt='%i',             # formatting, 2 digits in this case
    delimiter=',',          # column delimiter
    newline='\n')

# Census

In [ ]:
import csv
raw_census_data = []
numerical = ['age', 'fnlwgt','education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
with open('../data/census_data/train_data.csv') as csvFile:
    reader = csv.DictReader(csvFile)
    
    for row in reader:
        raw_census_data.append(row)
        

census_data = []
census_label =[]
seen = {}

for data in raw_census_data:
    dp = []
    for attr in data:
        if attr == "label":
            census_label.append(int(data[attr]))
            
        if data[attr] == '?':
            dp.append(0)
            
        if attr in numerical:
            dp.append(int(data[attr]))
        else:
            if attr not in seen:
                seen[attr] = []
            
            if data[attr] not in seen[attr]:
                seen[attr].append(data[attr])
            dp.append(seen[attr].index(data[attr]))
    
    census_data.append(dp)
    
typetable = [x == 0 for x in census_data[0]]
    
census_train_data = census_data    
census_label= np.array(census_label)

In [ ]:
import csv
raw_census_data = []
numerical = ['age', 'fnlwgt','education-num', 'capital-gain', 'capital-loss', 'hours-per-week' ]
with open('../data/census_data/test_data.csv') as csvFile:
    reader = csv.DictReader(csvFile)
    
    for row in reader:
        raw_census_data.append(row)
        

census_data = []

for data in raw_census_data:
    dp = []
    for attr in data:
        if attr == "label":
            census_label.extend(int(data[attr]))
            
        if data[attr] == '?':
            dp.append(0)
            
        if attr in numerical:
            dp.append(int(data[attr]))
        else:
            if attr not in seen:
                seen[attr] = []
            
            if data[attr] not in seen[attr]:
                seen[attr].append(data[attr])
            dp.append(seen[attr].index(data[attr]))
    
    census_data.append(dp)
    

census_test_data = census_data

In [ ]:
ents = rforest(census_train_data, census_label, 20, int(len(census_train_data)/5), maxdepth=20)
len(typetable)

In [ ]:
ents.train(typetable=typetable, heuristic=entropy_heuristic())

In [ ]:
print("Classification error!")
net_error = 0
for i in range(len(census_train_data)):
    pred = ents.classify(census_train_data[i]) 
    if pred is None:
        print(pred)
    if pred is not None:
        net_error += abs(pred - census_label[i])
    
print(net_error/len(spam_valid_data))

In [ ]:
spam_results = []


for i, dp in enumerate(census_test_data):
    pred = ents.classify(dp) 
    if pred is None:
        pred = 0
    spam_results.append(np.array([i+1, pred]))

np.savetxt(
    'census.csv',           # file name
    spam_results,                # array to save
    fmt='%i',             # formatting, 2 digits in this case
    delimiter=',',          # column delimiter
    newline='\n')